To add:
* lowering sentences seems to be helpful
* averaging XGBoost and LR seems to help slightly
* finding sentences for g + v separately is good, then use both
* for cutoffs, use nEmpty instead of len(merged).
* will use bigrams based on g (lower gene!)



In [1]:
import nltk
import numpy as np
import pandas as pd
import sklearn
import _pickle as pickle
import string
import os
from nltk.classify import NaiveBayesClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

In [22]:
%cd "/Users/hjohnsen/Desktop"

/Users/hjohnsen/Desktop


In [23]:
if not os.path.exists('processedmerged.pkl'):
    print("remaking merged")
    tvarsdf = pd.read_csv('training_variants', sep=",")
    tvarsdf["split"]=np.random.random([len(tvarsdf)])<0.8
    ttextdf = pd.read_csv('training_text', sep="\|\|")
    ttextdf.rename(columns={'ID,Text': "Text"}, inplace = True)
    ttextdf["ID"]=ttextdf.index
    merged = pd.merge(tvarsdf,ttextdf, how="inner")
    merged["Text"]=merged["Text"].str.lower()
    merged["tokenized"]=merged["Text"].map(nltk.word_tokenize)    
    merged["Gene"]=merged["Gene"].str.lower()
#    with open("tokenized.pkl",'wb') as f:
#        pickle.dump(merged, f)
else:
    merged=pickle.load(open('processedmerged.pkl','rb'))
    merged=merged[["ID","Gene","Text","Variation", "Class", "tokenized"]]

remaking merged


/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:5: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """


import time
if not os.path.exists('tokenized.pkl'):
    tvarsdf = pd.read_csv('training_variants', sep=",")
    tvarsdf["split"]=np.random.random([len(tvarsdf)])<0.8
    ttextdf = pd.read_csv('training_text', sep="\|\|")
    ttextdf.rename(columns={'ID,Text': "Text"}, inplace = True)
    ttextdf["ID"]=ttextdf.index
    merged = pd.merge(tvarsdf,ttextdf, how="inner")
    merged["tokenized"]=merged["Text"].map(nltk.word_tokenize)
    with open("tokenized.pkl",'wb') as f:
        pickle.dump(merged, f)
else:
    while True:
        try:
            merged=pickle.load(open('tokenized.pkl','rb'))
            merged=merged[["ID","Gene","Text","Variation", "Class", "tokenized"]]
            break
        except:
            time.sleep(600)

In [24]:
uniqueGenes = merged["Gene"].unique()

In [25]:
def build_bag_of_words_features_filtered(words):
    bag = {}
    useless_words = nltk.corpus.stopwords.words("english") + list(string.punctuation)
    for word in words:
        if not word in useless_words:
            bag[word]=1
    return bag

In [26]:
def build_bag_of_genes(words):
    bag={}
    for word in words:
        if word.upper() in uniqueGenes:
            bag[word.upper()]=1
    return bag

In [27]:
types = ["fusion", "del", "dup", "ins", "trunc", "splice", "\*"]
def findMutTypes(types, df):
    for i in types:
        df["is"+i] = df["Variation"].str.lower().str.contains(i).astype(int)

    df["mutType"]=list(zip(df["isfusion"], 
                           df["isdel"], 
                           df["isdup"], 
                           df["isins"], 
                           df["istrunc"], 
                           df["issplice"], 
                           df["is\*"]))
    df["mutType"]=df["mutType"].map(list)

    for i in types:
        del df["is"+i]

In [28]:
def getvSentences(gene, variation, text):
    sentences = ""
    splitText = text.split(".")
    for sentence in splitText:
        if variation.lower() in sentence.lower():
            sentences += sentence
    return sentences.lower()
    
def applyvGetSentences(row):
    gene = row["Gene"]
    variation = row["Variation"]
    text = row["Text"]
    return getvSentences(gene, variation, text)

def getgSentences(gene, variation, text):
    variation = gene
    sentences = ""
    splitText = text.split(".")
    for sentence in splitText:
        if variation.lower() in sentence.lower():
            sentences += sentence
    return sentences.lower()
    
def applygGetSentences(row):
    gene = row["Gene"]
    variation = row["Variation"]
    text = row["Text"]
    return getgSentences(gene, variation, text)

def find_bigrams(input_sentence):
    input_list = input_sentence.split()
    return list(zip(input_list, input_list[1:]))


def getabridgedwords(df):
    wordcounts = {}
    for i in df["tokenized"]:
        words = set(i)
        for word in words:
            wordcounts[word] = wordcounts.get(word,0)+1
    sortedwords = sorted(wordcounts.items(), key=lambda c: c[1], reverse=True)
    abridgedsorted = {}
    for (k,v) in sortedwords:
        if v< len(merged)*.9 and v>len(merged)*.1:
            abridgedsorted[k]=v
    return abridgedsorted


def getabridgedsentencewords(array):
    nEmpty = 0
    wordcounts = {}
    for i in array:
        if len(i)>0:        
            nEmpty+=1
        words = set(i)
        for word in words:
            wordcounts[word] = wordcounts.get(word,0)+1
    sortedwords = sorted(wordcounts.items(), key=lambda c: c[1], reverse=True)
    abridgedsentencewords = {}
    for (k,v) in sortedwords:
        if v< nEmpty*.9 and v>nEmpty*.1:
            abridgedsentencewords[k]=v
    return abridgedsentencewords



def getabridgedbigrams(df):
    flat_list = [item for sublist in df["bigrams"].tolist() for item in set(sublist)]
    bigs = {}
    for i in flat_list:
        bigs[i] = bigs.get(i,0)+1
    sortedbgs=sorted(bigs.items(), key=lambda c: c[1], reverse=True)
    abridgedsortedbgs = {}
    for (k,v) in sortedbgs:
        if v< len(df)*.5 and v>59:
            abridgedsortedbgs[k]=v
    return abridgedsortedbgs

In [33]:
def makeAllTheBags(df):
    df["wordbag"]=df["tokenized"].map(build_bag_of_words_features_filtered)
    print("wordbag made")
    df["genebag"]=df["tokenized"].map(build_bag_of_genes)
    print("genebag made")
    df["gSentences"] = df.apply(applygGetSentences, axis = 1)
    df["vSentences"] = df.apply(applyvGetSentences, axis = 1)
    print("sentences made")
    df["tokenizedgSentences"]=df["gSentences"].map(nltk.word_tokenize)
    df["tokenizedvSentences"]=df["vSentences"].map(nltk.word_tokenize)
    print("sentences tokenized")
    df["sentencegWordBag"]=df["tokenizedgSentences"].map(build_bag_of_words_features_filtered)
    df["sentencevWordBag"]=df["tokenizedvSentences"].map(build_bag_of_words_features_filtered)
    print("sentence wordbag made")
    df["bigrams"]=df["gSentences"].map(find_bigrams)
    print("bigrams made")
    
def makeAllTheFeatures(df):
    df["genesmentioned"]=df["genebag"].map(lambda b: [int(gene in b) for gene in uniqueGenes])
    print("genebag feature made")
    df["geneofinterest"]=df["Gene"].map(lambda b: [int(gene == b) for gene in uniqueGenes])
    print("goi made")
    df["wordfeatures"]=df["wordbag"].map(lambda b: [int(word in b.keys()) for word in abridgedsorted.keys()])
    print("wordfeatures made")
    findMutTypes(types, df)
    print("muttype features made")
    df["sentencegWordFeatures"]=df["sentencegWordBag"].map(lambda b: [int(word in b.keys()) for word in abridgedsentencegwords.keys()])
    df["sentencevWordFeatures"]=df["sentencevWordBag"].map(lambda b: [int(word in b.keys()) for word in abridgedsentencevwords.keys()])
    print("sentence word features made")
    df["bigramfeatures"]=df["bigrams"].map(lambda b: [int(word in b) for word in abridgedsortedbgs.keys()])
    print("bigram features made")
    
    df["all"]=df["wordfeatures"]+df["genesmentioned"]+df["geneofinterest"]+df["mutType"]+df["sentencegWordFeatures"]+df["sentencevWordFeatures"]+df["bigramfeatures"]


In [32]:
merged["all"]=merged["wordfeatures"]+merged["genesmentioned"]+merged["geneofinterest"]+merged["mutType"]+merged["sentencegWordFeatures"]+merged["sentencevWordFeatures"]+merged["bigramfeatures"]


In [30]:
makeAllTheBags(merged)

abridgedsorted=getabridgedwords(merged)
abridgedsentencegwords=getabridgedsentencewords(merged["sentencegWordBag"])
abridgedsentencevwords=getabridgedsentencewords(merged["sentencevWordBag"])
abridgedsortedbgs=getabridgedbigrams(merged)

makeAllTheFeatures(merged)

wordbag made
genebag made
sentences made
sentences tokenized
sentence wordbag made
bigrams made
genebag feature made
goi made
wordfeatures made
muttype features made
sentence word features made
bigram features made


KeyError: 'sentenceWordFeatures'

In [34]:
if not os.path.exists('tokenizedtest.pkl'):
    print("remaking testmerged")
    testvarsdf = pd.read_csv('stage2_test_variants.csv', sep=",")
    testtextdf = pd.read_csv('stage2_test_text.csv', sep="\|\|")
    testtextdf.rename(columns={'ID,Text': "Text"}, inplace = True)
    testtextdf["ID"]=testtextdf.index
    testmerged = pd.merge(testvarsdf,testtextdf, how="inner")
    testmerged["Text"]=testmerged["Text"].str.lower()    
    testmerged["tokenized"]=testmerged["Text"].map(nltk.word_tokenize)
    testmerged["Gene"]=testmerged["Gene"].str.lower()    
#    with open("tokenizedtest.pkl",'wb') as f:
#        pickle.dump(testmerged, f)
else:
    testmerged=pickle.load(open('tokenizedtest.pkl','rb'))

remaking testmerged


/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:4: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  after removing the cwd from sys.path.


In [35]:
makeAllTheBags(testmerged)
makeAllTheFeatures(testmerged)

wordbag made
genebag made
sentences made
sentences tokenized
sentence wordbag made
bigrams made
genebag feature made
goi made
wordfeatures made
muttype features made
sentence word features made
bigram features made


In [38]:
training = merged
test = testmerged
X_train = training["all"].tolist()
y_train = training["Class"].tolist()
X_test=test["all"].tolist()

lr = LogisticRegression( solver = "lbfgs",
                                multi_class="multinomial",
                                penalty='l2',
                                C=1e-2,
                                fit_intercept=True
                                )
lr.fit(X_train, y_train)
y_predlr = lr.predict_proba(X_test)

with open("submission.csv", "w") as out:
    out.write("ID,class1,class2,class3,class4,class5,class6,class7,class8,class9\n")
    for i,j in enumerate(y_pred):
        out.write(str(i+1)+",")
        out.write(",".join(map(str, j)))
        out.write("\n")

In [39]:
import xgboost as xgb
model = xgb.XGBClassifier(eval_metric="mlogloss", max_depth=6)
training = merged
test = testmerged
X_train = np.array(training["all"].tolist())
y_train = np.array(training["Class"].tolist())

model.fit(X_train, y_train)

X_test=test["all"].tolist()
y_predxgb = model.predict_proba(X_test) 


In [40]:
y_pred = ((y_predlr)+(y_predxgb))/2

with open("xgbsubmission.csv", "w") as out:
    out.write("ID,class1,class2,class3,class4,class5,class6,class7,class8,class9\n")
    for i,j in enumerate(y_pred):
        out.write(str(i+1)+",")
        out.write(",".join(map(str, j)))
        out.write("\n")

In [ ]:
#testmerged.describe()

In [ ]:
#testvarsdf.describe()

In [ ]:
#testtextdf.describe()

In [ ]:
#testmerged.head()

In [ ]:
import matplotlib.pyplot as pyplot
# plot
pyplot.bar(range(len(model.feature_importances_)), model.feature_importances_)
pyplot.show()


In [44]:
#df["wordfeatures"]+df["genesmentioned"]+df["geneofinterest"]+df["mutType"]+df["sentenceWordFeatures"]+df["bigramfeatures"]
wf = len(merged["wordfeatures"][0])
gm = len(merged["genesmentioned"][0])
gi = len(merged["geneofinterest"][0])
mt = len(merged["mutType"][0])
swf= len(merged["sentencegWordFeatures"][0])+len(merged["sentencevWordFeatures"][0])
bgf= len(merged["bigramfeatures"][0])

print("wordfeatures " + "0" + " to " + str(wf))
print("genesmentioned " + str(wf) + " to " + str(wf+gm))
print("geneofinterest " + str(wf+gm) + " to " + str(wf+gm+gi))
print("mutType " + str(wf+gm+gi) + " to " + str(wf+gm+gi+mt))
print("sentencegWordFeatures " + str(wf+gm+gi+mt) + " to " + str(wf+gm+gi+mt+swf))
print("bigramfeatures " + str(wf+gm+gi+mt+swf) + " to " + str(wf+gm+gi+mt+swf+bgf))


wordfeatures 0 to 3578
genesmentioned 3578 to 3842
geneofinterest 3842 to 4106
mutType 4106 to 4113
sentencegWordFeatures 4113 to 5930
bigramfeatures 5930 to 23668


pyplot.figure(figsize=(10,5))
pyplot.axvline(3862)
pyplot.axvline(4131)
pyplot.axvline(4400)
pyplot.axvline(4407)
pyplot.axvline(5013)
pyplot.bar(range(len(model.feature_importances_)), model.feature_importances_)
pyplot.show()

In [41]:
with open("processedmerged.pkl",'wb') as f:
    pickle.dump(merged, f)
with open("processedtestmerged.pkl",'wb') as f:
    pickle.dump(testmerged, f)

In [42]:
print(len(merged["all"][0]))

23668


In [48]:
merged["nobgs"]=merged["wordfeatures"]+merged["genesmentioned"]+merged["geneofinterest"]+merged["mutType"]+merged["sentencegWordFeatures"]+merged["sentencevWordFeatures"]

In [65]:
merged["split"]=np.random.random([len(merged)])<0.8


In [49]:
def logistictester(df, X, Y, C=1e-2):
    training = df[df["split"]]
    test = df[df["split"]==False]
    X_train = training[X].tolist()
    y_train = training[Y].tolist()
    X_test=test[X].tolist()

    lr = LogisticRegression( solver = "lbfgs",
                                multi_class="multinomial",
                                penalty='l2',
                                C=C,
                                fit_intercept=True
                                )
    lr.fit(X_train, y_train)
    y_pred = lr.predict_proba(X_test) 
    
    ls = []
    for i in range(len(y_pred)):
        c = test.iloc[i][Y]
        prob = y_pred[i][c-1]
        ls.append(-np.log(prob+1e-6))

    print("Accuracy on test set:")
    print(np.mean(ls))

    ytrain_pred = lr.predict_proba(X_train)
    ls = []
    for i in range(len(ytrain_pred)):
        c = training.iloc[i]["Class"]
        prob = ytrain_pred[i][c-1]
        ls.append(-np.log(prob+1e-6))
    print("Accuracy on training set:")
    print(np.mean(ls))

In [67]:
logistictester(merged, "all", "Class")
logistictester(merged, "nobgs", "Class")

Accuracy on test set:
0.920736077104
Accuracy on training set:
0.335322950712
Accuracy on test set:
0.919453301164
Accuracy on training set:
0.388750127138


In [51]:
del merged["bigrams"]

In [52]:
del testmerged

In [53]:
merged["bigrams"]=merged["vSentences"].map(find_bigrams)
print("found")
abridgedsortedbgs=getabridgedbigrams(merged)
print("abridged")
merged["bigramfeatures"]=merged["bigrams"].map(lambda b: [int(word in b) for word in abridgedsortedbgs.keys()])
print("features found")

found
abridged
features found


In [61]:
merged["plusvbgs"]= merged["nobgs"]+merged["bigramfeatures"]

In [66]:
logistictester(merged, "plusvbgs", "Class")


Accuracy on test set:
0.914128765578
Accuracy on training set:
0.371784775649


In [64]:
print(len(merged["all"][0]))
print(len(merged["bigramfeatures"][0]))
print(len(merged["plusvbgs"][0]))

23668
981
6911


In [2]:
subs=pd.read_csv("xgbsubmission.csv")

In [3]:
subs.head()

,ID,class1,class2,class3,class4,class5,class6,class7,class8,class9
0,1,0.057553,0.023231,0.005634,0.792881,0.013617,0.073030,0.030131,0.001729,0.002194
1,2,0.944951,0.002815,0.001411,0.034696,0.002216,0.002596,0.010579,0.000393,0.000343
2,3,0.059542,0.244398,0.004076,0.055069,0.013307,0.080165,0.535708,0.005366,0.002369
3,4,0.123668,0.130612,0.003017,0.151940,0.005809,0.038985,0.531747,0.009457,0.004765
4,5,0.009415,0.043537,0.005621,0.042695,0.015561,0.005647,0.874700,0.001110,0.001713


In [4]:
subs["class1"].sum()

172.99577381837216

In [8]:
for col in subs.columns[1:]:
    su=subs[col].sum()
    subs[col]=subs[col]/su

In [9]:
subs.head()

,ID,class1,class2,class3,class4,class5,class6,class7,class8,class9
0,1,0.000333,0.000133,0.000467,0.004561,0.000373,0.001215,0.000088,0.000290,0.000261
1,2,0.005462,0.000016,0.000117,0.000200,0.000061,0.000043,0.000031,0.000066,0.000041
2,3,0.000344,0.001404,0.000338,0.000317,0.000364,0.001334,0.001566,0.000901,0.000282
3,4,0.000715,0.000750,0.000250,0.000874,0.000159,0.000649,0.001555,0.001588,0.000567
4,5,0.000054,0.000250,0.000466,0.000246,0.000426,0.000094,0.002558,0.000186,0.000204


In [10]:
subs.to_csv("latesub.csv", inc)

In [11]:
subs["class1"].sum()

1.0000000000000007

In [12]:
subs.describe()

,ID,class1,class2,class3,class4,class5,class6,class7,class8,class9
count,986.000000,986.000000,986.000000,986.000000,986.000000,986.000000,986.000000,9.860000e+02,986.000000,986.000000
mean,493.500000,0.001014,0.001014,0.001014,0.001014,0.001014,0.001014,1.014199e-03,0.001014,0.001014
std,284.777984,0.001365,0.001145,0.005117,0.001301,0.003076,0.002098,8.694272e-04,0.005278,0.007955
min,1.000000,0.000002,0.000002,0.000013,0.000002,0.000013,0.000005,8.023569e-07,0.000014,0.000009
25%,247.250000,0.000153,0.000164,0.000166,0.000135,0.000164,0.000167,1.793319e-04,0.000268,0.000132
50%,493.500000,0.000477,0.000661,0.000299,0.000474,0.000288,0.000441,8.398229e-04,0.000518,0.000235
75%,739.750000,0.001142,0.001398,0.000486,0.001472,0.000597,0.000978,1.623307e-03,0.000894,0.000365
max,986.000000,0.005616,0.005625,0.068812,0.005686,0.025474,0.016216,2.906599e-03,0.120984,0.116505
